# Zip Code Median Income
_Calvin Whealton_

This notebook combines the data from the US Census American Community Survey 2018. The data were downloaded from the US Census website, specifically tables S1901 for each of the 48 contiguous states and Washington, DC.

Because the data is available at the county subdivision, but not the zip code, the values must be spatially disaggregated and summed. This is accomplished by using a file that maps between the zip code and the county subdivision.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import os

## Stitching State Data Together

This section merges the files together to make one file that is the median income. There are many more data fields that are not processed. The column for median income estimate for the county subdivision is S1901_C01_012E.

In [ ]:
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/us_census_s1901_income/csvs')

In [ ]:
state_incs_list = os.listdir()

In [ ]:
# dataframe for values of median income in each county subdivision
state_medinc_cousub = pd.DataFrame(columns=['GEOID_COUSUB','med_hh_inc'])

In [ ]:
# loop over all files (states)
for file in state_incs_list:
    
    # row 1 has explanation/long captions for column titles
    state_inc = pd.read_csv(file,skiprows=[1])
    
    # dictionary of values from the state
    state_med = pd.DataFrame({'GEOID_COUSUB':state_inc['GEO_ID'],
                              'med_hh_inc':state_inc['S1901_C01_012E']})
    
    # appending values to the large state dataframe
    state_medinc_cousub = state_medinc_cousub.append(state_med,ignore_index=True)

Doing some basic checks and post-processing

In [ ]:
state_medinc_cousub.shape

In [ ]:
state_medinc_cousub.isnull().sum(axis=0).sum()

In [ ]:
state_medinc_cousub = state_medinc_cousub.dropna

In [ ]:
state_medinc_cousub.head()

In [ ]:
state_medinc_cousub['GEOID_CS'] = [string.split('US')[1] for string in state_medinc_cousub['GEOID_COUSUB']]

In [ ]:
state_medinc_cousub.head()

In [ ]:
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/processed_data')
state_medinc_cousub.to_csv('state_medinc_cousub.csv')

# Aggregating/Disaggregating to Zip Code

The median household income for a zip code will be assigned as the median income based on a population-weighted median income for the zip codes that share the value. For example, if zip code 12345 is composed of 40% of the population from county subdivision 23 with median income 10,000 and 60%  from county subdivision 45 with median income 20,000, then the estimated median income for the zip code will be 10,000x0.4 + 20,000x0.6 = 4,000 + 12,000 = 16,000. In the case where some county subdivisions do not have data, the fractions are normalized to sum to 1.

In [ ]:
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data')

zcta_cousub_map = pd.read_csv('zcta_countysub_uscensus.txt')

In [ ]:
zcta_cousub_map.head()

In [ ]:
zip_med_inc = pd.DataFrame(columns=['zip','med_hh_inc'])

In [ ]:
# unique values of zip code
zip_use = zcta_cousub_map.ZCTA5.unique()


# loop over every zip code
for zc in zip_use:
    
    # used in calculations for each zip code
    temp_df = pd.DataFrame()
    
    # geoids for the county
    # casting to string to match string in other dataframe
    cou_geoids = zcta_cousub_map.loc[zcta_cousub_map['ZCTA5'] == zc,'GEOID'].astype(str).values
    cou_df = zcta_cousub_map.loc[zcta_cousub_map['ZCTA5'] == zc]
    
    # searching for geoids in the median income data
    temp_df = state_medinc_cousub.loc[state_medinc_cousub['GEOID_CS'].isin(cou_geoids)]
    
    if temp_df.shape[0] == 0:
        zip_med_inc = zip_med_inc.append({'zip':zc, 'med_hh_inc': np.NaN},ignore_index=True)
        
    else:
        
        # adding column for population fraction [0,100]
        temp_df['pop_frac'] = 0
        
        #finding the fractions that match the county subdivisions
        for i in temp_df.index:
            temp_df.loc[i,'pop_frac'] = cou_df.loc[cou_df['GEOID'].values.astype(str) == temp_df.loc[i,'GEOID_CS'],'ZPOPPCT'].values
        
        # empty values removed
        temp_df = temp_df[temp_df['med_hh_inc'] != '-']
        
        # special cases for the boundary values of income
        # only listed as above or below a value
        for i in temp_df.index:
            if temp_df.loc[i,'med_hh_inc'] == '250,000+':
                temp_df.loc[i,'med_hh_inc'] = '250000'
            if temp_df.loc[i,'med_hh_inc'] == '2,500-':
                temp_df.loc[i,'med_hh_inc'] = '2500'
        
        # estimate of the median income
        est_med_inc = np.sum(np.array(temp_df['pop_frac'].values)*np.array(temp_df['med_hh_inc'].values.astype(float)))/(np.sum(np.array(temp_df['pop_frac'].values)))
        
        zip_med_inc = zip_med_inc.append({'zip':zc, 'med_hh_inc': est_med_inc},ignore_index=True)

Basic checks and cleaning following operation

In [ ]:
zip_med_inc.head()

In [ ]:
zip_med_inc.reset_index()

In [ ]:
zip_med_inc.shape[0]

In [ ]:
zip_med_inc = zip_med_inc.dropna()

In [ ]:
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/processed_data')
zip_med_inc.to_csv('zips_med_inc.csv')